<a href="https://colab.research.google.com/github/maheravi/Deep-Learning/blob/main/PyTorch%20Persian%20Mnist%20TL/Persian_Mnist_Sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb --upgrade

In [2]:
import wandb

wandb.login()

wandb: Currently logged in as: ma_heravi (use `wandb login --relogin` to force relogin)


True

In [15]:
import torch
from torch import nn
import torchvision
from torch.utils.data import DataLoader

In [16]:
sweep_config = {
    'method': 'random'
    }

In [17]:
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [18]:
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'fc_layer_size': {
        'values': [128, 256, 512]
        },
    'dropout': {
          'values': [0.3, 0.4, 0.5]
        },
    }

sweep_config['parameters'] = parameters_dict

In [19]:
parameters_dict.update({
    'epochs': {
        'value': 1}
    })

In [20]:
import math

parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform',
        'q': 1,
        'min': math.log(32),
        'max': math.log(256),
      }
    })

In [21]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform',
                               'max': 5.545177444479562,
                               'min': 3.4657359027997265,
                               'q': 1},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'value': 1},
                'fc_layer_size': {'values': [128, 256, 512]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [27]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-PesianMnist")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: cj8vwf1f
Sweep URL: https://wandb.ai/ma_heravi/pytorch-sweeps-PesianMnist/sweeps/cj8vwf1f


In [28]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        loader = build_dataset(config.batch_size)
        network = build_network(config.fc_layer_size, config.dropout)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        for epoch in range(1, epochs+1):
            total_loss = train_epoch(network, loader, optimizer)
            print(f"Epoch: {epoch+1}, Loss: {total_loss}")
            wandb.log({'epochs':  epoch + 1,
              'loss': total_loss,
              'acc': total_acc
                              })           

In [29]:
def build_dataset(batch_size):
   
    transform = torchvision.transforms.Compose([
            torchvision.transforms.Resize((28, 28)),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])


    dataset = torchvision.datasets.ImageFolder("/content/drive/MyDrive/MNIST_persian", transform=transform)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return loader


def build_network(fc_layer_size, dropout):
    network = models.resnet50(pretrained=True)
    in_fetures = network.fc.in_features
    network.fc = nn.Linear(in_fetures, 1)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # This freezes layers 1-6 in the total 10 layers of Resnet50
    ct = 0
    for child in network.children():
        ct += 1
        if ct < 7:
            for param in child.parameters():
                param.requires_grad = False
                
    return network.to(device)
        

def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


In [30]:
def train_epoch(network, loader, optimizer):

    train_loss = 0.0
    train_acc = 0.0
    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        # 1- forwarding
        preds = network(images)
        # 2- backwarding 
        loss = loss_function(preds, labels)
        loss.backward()
        # 3- Update
        optimizer.step()

        train_loss += loss
        train_acc += calc_acc(preds, labels)
    
    total_loss = train_loss / len(loader)
    total_acc = train_acc / len(loader)

In [31]:
wandb.agent(sweep_id, train)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: xz1xy36m with config:
wandb: 	batch_size: 47
wandb: 	dropout: 0.5
wandb: 	epochs: 1
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.01755756874335218
wandb: 	optimizer: sgd


wandb: Processing terminal ouput (stdout)...
wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
torch.save(model.state_dict(), "PersianMnistSweepTL.pth")